In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.edge.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from time import sleep
import pandas as pd

In [4]:
# Create a function which pull window

def initialize_driver():
    path = r"C:\Users\vanda\Downloads\edgedriver_win32 (1)\msedgedriver.exe"
    service = Service(executable_path=path)
    driver = webdriver.Edge(service=service)
    return driver

In [10]:
def scrape_data(driver, url):
    driver.get(url)
    
    Product_title = []
    Product_price = []
    EAN = []

    for i in range(7):
        print('scraping page', i + 1)
        
        # Wait for elements to load
        WebDriverWait(driver, 50).until(
            EC.presence_of_element_located((By.XPATH, '//a[@role="heading"]'))
        )

        # Finding Product Price
        price_elements = driver.find_elements(By.XPATH, '//span[@data-test="price"]')
        Product_price.extend([p.text for p in price_elements])

        # Find product name
        name_elements = driver.find_elements(By.XPATH, '//a[@role="heading"]')
        Product_title.extend([n.text for n in name_elements])

        # Finding the EAN code
        name_elements = driver.find_elements(By.XPATH, "//dt[contains(@class, 'specs_title') and text()='EAN']/following-sibling::dd[contains(@class, 'specs_value')]")
        EAN.extend([n.text for n in name_elements])

        # Navigate to next page
        try:
            next_button = WebDriverWait(driver,40).until(
                EC.element_to_be_clickable((By.XPATH, '//a[@class="ui-btn  ui-btn--secondary ui-btn--square"]'))
            )
            next_button.click()
            sleep(30)
        except Exception as e:
            print(f"Error navigating to next page: {e}")
            break
    # Ensure all lists have the same length
    max_length = max(len(Product_price), len(Product_title), len(EAN))
    Product_price.extend([''] * (max_length - len(Product_price)))
    Product_title.extend([''] * (max_length - len(Product_title)))
    EAN.extend([''] * (max_length - len(EAN)))
    
    # Return the scraped data
    return {
        "Product_title": Product_title,
        "Product_price": Product_price,
        "EAN": EAN
    }


In [12]:
url = "https://www.bol.com/nl/nl/w/alle-artikelen-easyup/1817090/?sort=wishListRank1"
driver = initialize_driver()

try:
    Alle_artikelen_van_EasyUp_data = scrape_data(driver, url)
    Alle_artikelen_van_EasyUp_data_df = pd.DataFrame(Alle_artikelen_van_EasyUp_data)
    print(Alle_artikelen_van_EasyUp_data_df.head())
    Alle_artikelen_van_EasyUp_data_df.to_csv('Alle_artikelen_van_EasyUp.csv', index=False)
    print("Data saved to Alle_artikelen_van_EasyUp.csv")
finally:
    driver.quit()

scraping page 1
scraping page 2
scraping page 3
scraping page 4
scraping page 5
scraping page 6
scraping page 7


ProtocolError: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None))